<a href="https://colab.research.google.com/github/hksantosh/sscm/blob/main/ASCP/Production_planning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>